In [1]:
""""
import torch
from torch import nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader
#import numpy as np
#from tqdm import tqdm, tqdm_notebook
"""""
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2070'

In [3]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
##GPU 사용 시
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [7]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [8]:
import pandas as pd
import re

In [9]:
df_data1 = pd.read_csv('./output/sample2/sample1.csv')
df_data2 = pd.read_csv('./output/sample2/sample2.csv')
df_data3 = pd.read_csv('./output/sample2/sample3.csv')
df_data4 = pd.read_csv('./output/sample2/sample4.csv')
df_data5 = pd.read_csv('./output/sample2/sample5.csv', encoding = 'cp949')

In [10]:
df_data_p = pd.concat([df_data1, df_data2, df_data3,df_data4, df_data5])

In [9]:
use_col = ['train','label']

In [10]:
df_data = pd.read_csv('./labeling/data/data.csv')

In [11]:
df_data

,train,label
0,사설 北 핵·ICBM 시험 재개 시사… ‘잘못된 길’ 선택해선 안 된다김정은 노동당...,1
1,사설 수출 역성장 멈추려면 기업 활력부터 되살려내야지난해 수출이 10년 만에 두 자...,1
2,사설 여권 일방통행식 국회 운영 정국 정상화 가능하겠나새해 벽두부터 국회에 짙은 전...,2
3,설왕설래 인구절벽출산율이 높던 시절 우리나라는 적극적인 산아제한 정책으로 인구증가를...,0
4,세계포럼 친중 정책 레드라인 넘지 말아야北제재 완화 동조·中핵우산 발언 한·미 동맹...,1
...,...,...
5394836,택배현장 방문한 이재갑 고용노동부 장관김포 연합뉴스 김주성 기자 이재갑 고용노동부 ...,2
5394837,평택제천고속도로 ‘평택 복합휴게소’ 24일 개장화물차 많은 노선특성 반영 휴게소에 ...,1
5394838,이개호 발의 고향사랑기부금법 국회 행안위 통과지자체 기부금 모집 허용…농촌경제 활성...,2
5394839,화성시 관내 기업 제품 우선구매···지역경제 활성화서울경제 화성시는 지역 기업의 발...,2


In [ ]:
# df_data = df_data[['train','label']]

In [8]:
df_data.to_json('./data/data2.json', orient='records',  force_ascii=False)

MemoryError: 

In [12]:
my_client = MongoClient('mongodb://localhost:27017/')
newsDB = my_client['newsDB']
nosummary3 = newsDB['nosummary3']
data = nosummary3.insert_many(js)

3357775

In [12]:
df_data.sample(n=100)

,train,label
4997752,대학찰옥수수 판촉활동하는 이차영 괴산군수괴산 뉴시스 강신욱 기자 이차영 충북 괴산군...,2
2522433,김부겸 당대표 2년 임기 완벽 수행…대선 승리로 이끌 것25일 제주지역 더불어민주당...,2
3338457,후추·계피·큐민에서 쇳가루 검출…기준치 최대 18배 초과후추와 계피 큐민 등 시중에...,2
689106,사진꽃다발 받는 당대표들머니투데이 홍봉진 기자 바른미래당 손학규 대안신당 최경환 민...,2
567986,中서 신종코로나 억제 약물 발견…에이즈 치료제 일부 부작용 커신종 코로나바이러스 환...,2
...,...,...
2942084,BTS 출연한 서울에서 만나요 영상 공개서울 뉴시스 서울시와 서울관광재단 대표이사 ...,0
4536117,오비맥주 홍보·법조 부서 통합···수장은 구자범 부사장서울 뉴시스 오비맥주가 ‘여성...,2
1369395,소방청 차장 한국소방산업기술원 소방장비센터 현장 점검서울 연합뉴스 김홍필 소방청 차...,2
2454621,왕초보영어탈출 해커스톡 하루 1분 기초 영어회화 3월 23일Misery loves ...,2


In [14]:
df_data_p.loc[(df_data_p['label'] == "하락"), 'label'] = 1  #하락 => -1
df_data_p.loc[(df_data_p['label'] == "중립"), 'label'] = 0  #중립 => 0
df_data_p.loc[(df_data_p['label'] == "상승"), 'label'] = 2  #상승 => 1

In [13]:
df_data.loc[(df_data['label'] == "하락"), 'label'] = 1  #하락 => -1
df_data.loc[(df_data['label'] == "중립"), 'label'] = 0  #중립 => 0
df_data.loc[(df_data['label'] == "상승"), 'label'] = 2  #상승 => 1

In [14]:
df_data

,train,label
0,사설 北 핵·ICBM 시험 재개 시사… ‘잘못된 길’ 선택해선 안 된다김정은 노동당...,1
1,사설 수출 역성장 멈추려면 기업 활력부터 되살려내야지난해 수출이 10년 만에 두 자...,1
2,사설 여권 일방통행식 국회 운영 정국 정상화 가능하겠나새해 벽두부터 국회에 짙은 전...,2
3,설왕설래 인구절벽출산율이 높던 시절 우리나라는 적극적인 산아제한 정책으로 인구증가를...,0
4,세계포럼 친중 정책 레드라인 넘지 말아야北제재 완화 동조·中핵우산 발언 한·미 동맹...,1
...,...,...
5394836,택배현장 방문한 이재갑 고용노동부 장관김포 연합뉴스 김주성 기자 이재갑 고용노동부 ...,2
5394837,평택제천고속도로 ‘평택 복합휴게소’ 24일 개장화물차 많은 노선특성 반영 휴게소에 ...,1
5394838,이개호 발의 고향사랑기부금법 국회 행안위 통과지자체 기부금 모집 허용…농촌경제 활성...,2
5394839,화성시 관내 기업 제품 우선구매···지역경제 활성화서울경제 화성시는 지역 기업의 발...,2


In [17]:
data_list_p = []
for q, label in zip(df_data_p['제목'], df_data_p['label'])  :
    data = []
    clean_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', ' ', str(q)) 
    data.append(clean_title)
    data.append(str(label))

    data_list_p.append(data)

In [15]:
data_list = []
for q, label in zip(df_data['train'], df_data['label'])  :
    data = []
    clean_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', ' ', str(q)) 
    data.append(clean_title)
    data.append(str(label))

    data_list.append(data)

In [19]:
print(data_list_p[0])
print(data_list_p[800])

['대검 MBC·채널A에 녹음파일 제출하라 협조공문', '0']
['임한솔 전두환 불법재산 실마리 발견 검찰 수사 촉구', '1']


In [17]:
#데이터 셋 test와 train으로 4:1분리
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [18]:
print(len(dataset_train))
print(len(dataset_test))

4046130
1348711


In [19]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [36]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 1000
learning_rate =  4e-5

In [37]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [38]:
tok

In [26]:
data_predict = BERTDataset(data_list_p, 0, 1, tok, max_len, True, False)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


In [8]:
data_train[3]

NameError: name 'data_train' is not defined

In [28]:
predict_dataloader = torch.utils.data.DataLoader(data_predict, batch_size=1, num_workers=0)

In [23]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [24]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [36]:
model = torch.load('./model/model_1.pt').to(device)

In [25]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [26]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [27]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [28]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [29]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [30]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    # print(max_vals, max_indices,Y)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [32]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    predict_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    #torch.save(model, './model/model_{}.pt'.format(e+1))
    
    # model.eval()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(predict_dataloader)):
    #     token_ids = token_ids.long().to(device)
    #     segment_ids = segment_ids.long().to(device)
    #     valid_length= valid_length
    #     label = label.long().to(device)
    #     out = model(token_ids, valid_length, segment_ids)
    #     predict_acc += calc_accuracy(out, label)
    # print("epoch {} predict acc {}".format(e+1, predict_acc / (batch_id+1)))
    torch.save(model, './model/model_5_{}.pt'.format(e+1))
    

C:\Users\jeon\anaconda3\envs\stockpredict\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/63221 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.4078245460987091 train acc 0.78125
epoch 1 batch id 1001 loss 0.3161277174949646 train acc 0.8265328421578422
epoch 1 batch id 2001 loss 0.4636172950267792 train acc 0.8256028235882059
epoch 1 batch id 3001 loss 0.5894972085952759 train acc 0.8259798817060979
epoch 1 batch id 4001 loss 0.2942734360694885 train acc 0.8261215946013497
epoch 1 batch id 5001 loss 0.25687167048454285 train acc 0.8264003449310138
epoch 1 batch id 6001 loss 0.5552597641944885 train acc 0.8270158098650225
epoch 1 batch id 7001 loss 0.3905368149280548 train acc 0.8271854020854164
epoch 1 batch id 8001 loss 0.5926773548126221 train acc 0.8275039838770154
epoch 1 batch id 9001 loss 0.2945108711719513 train acc 0.8275764498389068
epoch 1 batch id 10001 loss 0.48901957273483276 train acc 0.8279515798420158
epoch 1 batch id 11001 loss 0.42293494939804077 train acc 0.8281221593491501
epoch 1 batch id 12001 loss 0.3537590503692627 train acc 0.8284557016081994
epoch 1 batch id 13001 loss 0.259

C:\Users\jeon\anaconda3\envs\stockpredict\lib\site-packages\ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/21074 [00:00<?, ?it/s]

epoch 1 test acc 0.804711130862431


  0%|          | 0/63221 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.3755180835723877 train acc 0.859375
epoch 2 batch id 1001 loss 0.39793169498443604 train acc 0.8203983516483516
epoch 2 batch id 2001 loss 0.47018685936927795 train acc 0.8185516616691654
epoch 2 batch id 3001 loss 0.5478614568710327 train acc 0.8184927940686438
epoch 2 batch id 4001 loss 0.2984071373939514 train acc 0.8186859535116221
epoch 2 batch id 5001 loss 0.3014809787273407 train acc 0.8188830983803239
epoch 2 batch id 6001 loss 0.5613365173339844 train acc 0.8190483877687053
epoch 2 batch id 7001 loss 0.43847647309303284 train acc 0.8191865447793173
epoch 2 batch id 8001 loss 0.5720211863517761 train acc 0.8190929258842645
epoch 2 batch id 9001 loss 0.3656715750694275 train acc 0.8188551966448172
epoch 2 batch id 10001 loss 0.47724294662475586 train acc 0.818910296470353
epoch 2 batch id 11001 loss 0.389248251914978 train acc 0.8189255522225253
epoch 2 batch id 12001 loss 0.331900417804718 train acc 0.8188614490459128
epoch 2 batch id 13001 loss 0.3056

  0%|          | 0/21074 [00:00<?, ?it/s]

epoch 2 test acc 0.8025308368496361


In [38]:
model = torch.load('./model/model_1.pt').to(device)

In [196]:
# model = BERTClassifier(torch.load('./model/model_2.pt'),dr_rate=0.5).to(device)

In [47]:
test_acc = 0.0
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(predict_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
print("epoch {} test acc {}".format(1, test_acc / (batch_id+1)))
#torch.save(model, './model/model_{}.pt'.format(e+1))

C:\Users\jeon\anaconda3\envs\stockpredict\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/1000 [00:00<?, ?it/s]

tensor([3.3722], device='cuda:0', grad_fn=<MaxBackward0>) tensor([0], device='cuda:0') tensor([0], device='cuda:0')
tensor([6.8688], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([6.8801], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([1], device='cuda:0')
tensor([5.6474], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([0], device='cuda:0')
tensor([6.0642], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([4.4677], device='cuda:0', grad_fn=<MaxBackward0>) tensor([0], device='cuda:0') tensor([2], device='cuda:0')
tensor([4.6987], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([2], device='cuda:0')
tensor([5.8723], device='cuda:0', grad_fn=<MaxBackward0>) tensor([1], device='cuda:0') tensor([0], device='cuda:0')
tensor([4.4263], device='cuda:0', grad_fn=<MaxBackward0>) tensor([0], de

In [ ]:
import pymongo